<a href="https://colab.research.google.com/github/angelkp570/CursoSpark/blob/master/Clase3_InferenciaTipoDatos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creación de un DataFrame a través de un RDD


In [ ]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment.
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"


# install findspark using pip
!pip install -q findspark

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, Row

from pyspark.sql import SQLContext

In [ ]:
from pyspark.sql.functions import *

In [ ]:
# ConfigureSparkUI
conf = SparkConf().set('spark.ui.port', '4050')

In [ ]:
sc = SparkContext(master="local", appName="DataFrames",conf=conf)

In [ ]:
sqlContext = SQLContext(sc)

# Carga del archivo deportista y deportista2

In [ ]:
deportistaOlimpicoRDD = sc.textFile("deportista.csv").map(lambda l : l.split(","))
deportistaOlimpicoRDD2 = sc.textFile("deportista2.csv").map(lambda l : l.split(","))

deportistaOlimpicoRDD = deportistaOlimpicoRDD.union(deportistaOlimpicoRDD2)

In [ ]:
deportistaOlimpicoRDD.take(5)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278']]

Crearemos un DataFrame a través del RDD deportistaOlimpicoRDD pero como poodemos observar este RDD contiene un encabezado. Por esta razón crearemos una función para poder retirar dicho encabezado.

Esta función va a recibir dos parámetros, indice e iterador. Esta función va a retornar una lista limpia que ya posee los valores que nosotros queremos.

iter() nos devuelve valor a valor lo que nosotros procesamos, aseguramos que devuelva una lista y que sea desde el segundo valor del RDD con esto eliminamos el encabezado  

In [ ]:
def eliminaEncabezado(indice, iterador):
  return iter(list(iterador)[1:])

#Ejemplo con función iter()

La función iter() en Python se utiliza para obtener un iterador a partir de un objeto iterable. Un iterador es un objeto que permite recorrer los elementos de un iterable, como una lista, tupla o cadena de caracteres, uno a la vez. Puedes usar la función iter() junto con la función next() para obtener y avanzar a través de los elementos del iterable.

Aquí hay un ejemplo sencillo que ilustra el uso de iter():

In [ ]:
# Crear una lista
mi_lista = [1, 2, 3, 4, 5]

# Obtener un iterador a partir de la lista
mi_iterador = iter(mi_lista)

# Obtener y mostrar el primer elemento del iterador
primer_elemento = next(mi_iterador)
print("Primer elemento:", primer_elemento)

# Obtener y mostrar el segundo elemento del iterador
segundo_elemento = next(mi_iterador)
print("Segundo elemento:", segundo_elemento)

Primer elemento: 1
Segundo elemento: 2


En este ejemplo:

1. Se crea una lista llamada mi_lista.
2. Se obtiene un iterador a partir de la lista utilizando la función iter().
3. Se utiliza la función next() para obtener el primer elemento del iterador y se imprime.
4. Se utiliza la función next() nuevamente para obtener el segundo elemento del iterador y se imprime.

Este es un ejemplo básico, pero la función iter() se utiliza comúnmente en situaciones donde se necesita trabajar con iteradores y se desea controlar manualmente el avance a través de los elementos de un iterable.

#Ejemplo con mapPartitionsWithIndex

La función mapPartitionsWithIndex es una transformación en Spark que permite aplicar una función a cada partición de un RDD, mientras proporciona el índice de la partición como un parámetro a la función. Esto es útil cuando necesitas realizar operaciones que dependen del índice de la partición en el que se está trabajando.

La sintaxis básica de mapPartitionsWithIndex es la siguiente:

<code>RDD.mapPartitionsWithIndex(función)</code>

Donde función es una función que toma dos argumentos: el índice de la partición y un iterador que produce los elementos de esa partición. La función devuelve un nuevo iterador que genera los elementos transformados.

Un ejemplo para ilustrar cómo funciona mapPartitionsWithIndex:


In [ ]:
#from pyspark import SparkContext

# Configuración del SparkContext
#sc = SparkContext("local", "Ejemplo Spark")

# Crear un RDD con datos de ejemplo
data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
rdd = sc.parallelize(data, 3)  # Crear un RDD con 3 particiones


# Definir una función para multiplicar cada elemento por el índice de la partición
def multiplicar_por_indice(partition_index, iterador):
    yield f"Partición {partition_index}: {list(iterador)}"

# Aplicar mapPartitionsWithIndex
resultado = rdd.mapPartitionsWithIndex(multiplicar_por_indice)

# Recoger y mostrar los resultados
resultados = resultado.collect()
for resultado_particion in resultados:
    print(resultado_particion)

Partición 0: [1, 2, 3]
Partición 1: [4, 5, 6]
Partición 2: [7, 8, 9, 10]


In [ ]:
rdd.take(5)

[1, 2, 3, 4, 5]

En este ejemplo:

1. Se crea un RDD con datos de ejemplo y se especifica que tenga 3 particiones.
2. Se define una función multiplicar_por_indice que toma el índice de la partición y un iterador que produce los elementos de esa partición. La función multiplica cada elemento por el índice de la partición.
3. Se aplica mapPartitionsWithIndex al RDD utilizando la función definida.
4. Se recogen y muestran los resultados, donde cada partición y sus elementos transformados se imprimen.

#mapPartitionsWithIndex
Esta función le pasa dos parámetros a nuestra función creada uno sería el índice y el otro toda la columna

In [ ]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.mapPartitionsWithIndex(eliminaEncabezado)

El resultado se nos muestra sin encabezado

In [ ]:
deportistaOlimpicoRDD.take(5)

[['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278'],
 ['5', 'Christine Jacoba Aaftink', '2', '21', '185', '82', '705']]

Antes de transformar el RDD primero tenemos que transformar los valores del RDD.

Por eso haremos un mapeo sobre deportistaOlimpicoRDD y tranformamos los valores

In [ ]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.map(lambda l: (
    int(l[0]),
    l[1],
    int(l[2]),
    int(l[3]),
    int(l[4]),
    float(l[5]),
    int(l[6])
    ))

In [ ]:
deportistaOlimpicoRDD.take(5)

[(1, 'A Dijiang', 1, 24, 180, 80.0, 199),
 (2, 'A Lamusi', 1, 23, 170, 60.0, 199),
 (3, 'Gunnar Nielsen Aaby', 1, 24, 0, 0.0, 273),
 (4, 'Edgar Lindenau Aabye', 1, 34, 0, 0.0, 278),
 (5, 'Christine Jacoba Aaftink', 2, 21, 185, 82.0, 705)]

Generamos el esquema que va a contener nuestro DataFrame

In [ ]:
schema = StructType([
    StructField("deportista_id", IntegerType(), False),
    StructField("nombre", StringType(), False),
    StructField("genero", IntegerType(), False),
    StructField("edad", IntegerType(), False),
    StructField("altura", IntegerType(), False),
    StructField("peso", FloatType(), False),
    StructField("equipo_id", IntegerType(), False)
])

In [ ]:
sqlContext.createDataFrame(deportistaOlimpicoRDD, schema).show(5)

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            1|           A Dijiang|     1|  24|   180|80.0|      199|
|            2|            A Lamusi|     1|  23|   170|60.0|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|     0| 0.0|      273|
|            4|Edgar Lindenau Aabye|     1|  34|     0| 0.0|      278|
|            5|Christine Jacoba ...|     2|  21|   185|82.0|      705|
+-------------+--------------------+------+----+------+----+---------+
only showing top 5 rows



In [ ]:
deportistaDF = sqlContext.createDataFrame(deportistaOlimpicoRDD, schema)

#Operaciones con los DataFrames

Primero cargamos los otros archivos en dataFrames

## paises.csv


In [ ]:
paisesRDD = sc.textFile("paises.csv").map(lambda l: l.split(","))
paisesRDD = paisesRDD.mapPartitionsWithIndex(eliminaEncabezado)
paisesRDD.take(5)

[['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG'],
 ['5', 'Afghanistan', 'AFG']]

In [ ]:
paisesRDD = paisesRDD.map(lambda l: (
    int(l[0]),
    l[1],
    l[2]
))

schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("equipo", StringType(), False),
    StructField("sigla", StringType(), False)
])

paisesDF = sqlContext.createDataFrame(paisesRDD, schema)

In [ ]:
paisesDF.show(5)

+---+--------------------+-----+
| id|              equipo|sigla|
+---+--------------------+-----+
|  1|         30. Februar|  AUT|
|  2|A North American ...|  MEX|
|  3|           Acipactli|  MEX|
|  4|             Acturus|  ARG|
|  5|         Afghanistan|  AFG|
+---+--------------------+-----+
only showing top 5 rows



## resultados.csv

In [ ]:
"""resultadoRDD = sc.textFile("resultados.csv").map(lambda line: line.split(","))
resultadoRDD = resultadoRDD.mapPartitionsWithIndex(eliminaEncabezado)

resultadoRDD = resultadoRDD.map(lambda l: (
    int(l[0]),
    l[1],
    int(l[2]),
    int(l[3]),
    int(l[4])
))

schema = StructType([
    StructField("resultado_id", IntegerType(), False),
    StructField("medalla", StringType(), False),
    StructField("deportista_id", IntegerType(), False),
    StructField("juego_id", IntegerType(), False),
    StructField("evento_id", IntegerType(), False)
])

resultadoDF = sqlContext.createDataFrame(resultadoRDD, schema)"""

'resultadoRDD = sc.textFile("resultados.csv").map(lambda line: line.split(","))\nresultadoRDD = resultadoRDD.mapPartitionsWithIndex(eliminaEncabezado)\n\nresultadoRDD = resultadoRDD.map(lambda l: (\n    int(l[0]),\n    l[1],\n    int(l[2]),\n    int(l[3]),\n    int(l[4])\n))\n\nschema = StructType([\n    StructField("resultado_id", IntegerType(), False),\n    StructField("medalla", StringType(), False),\n    StructField("deportista_id", IntegerType(), False),\n    StructField("juego_id", IntegerType(), False),\n    StructField("evento_id", IntegerType(), False)\n])\n\nresultadoDF = sqlContext.createDataFrame(resultadoRDD, schema)'

In [ ]:
#resultadoDF = resultadoDF.filter((resultadoDF.evento_id != '#N/A'))

In [ ]:
#resultadoDF.show(5)

In [ ]:
#resultadoDF[resultadoDF['evento_id']== 1].show()

In [ ]:
#resultadoDF[resultadoDF['evento_id']== "#N/A"].show()

In [ ]:
#resultadoDF[resultadoDF["resultado_id"]==251].take(1)

In [ ]:
resultadoSchema = StructType([
    StructField("resultado_id",IntegerType(),False),
    StructField("medalla",StringType(),False),
    StructField("deportista_id",IntegerType(),False),
    StructField("juego_id",IntegerType(),False),
    StructField("evento_id",IntegerType(),False)
])
resultadoDF = sqlContext.read.schema(resultadoSchema).option("header","true").csv("resultados.csv")

In [ ]:
resultadoDF.show(5)

+------------+-------+-------------+--------+---------+
|resultado_id|medalla|deportista_id|juego_id|evento_id|
+------------+-------+-------------+--------+---------+
|           1|     NA|            1|      39|        1|
|           2|     NA|            2|      49|        2|
|           3|     NA|            3|       7|        3|
|           4|   Gold|            4|       2|        4|
|           5|     NA|            5|      36|        5|
+------------+-------+-------------+--------+---------+
only showing top 5 rows



## juegos.csv

In [ ]:
juegoRDD = sc.textFile("juegos.csv").map(lambda line: line.split(","))
juegoRDD = juegoRDD.mapPartitionsWithIndex(eliminaEncabezado)

juegoRDD = juegoRDD.map(lambda l: (
    int(l[0]),
    l[1],
    l[2],
    l[3],
    l[4]
))

schema = StructType([
    StructField("juego_id", IntegerType(), False),
    StructField("nombre_juego", StringType(), False),
    StructField("anio", StringType(), False),
    StructField("temporada", StringType(), False),
    StructField("ciudad", StringType(), False)
])

juegoDF = sqlContext.createDataFrame(juegoRDD, schema)

In [ ]:
juegoDF.show(5)

+--------+------------+----+---------+---------+
|juego_id|nombre_juego|anio|temporada|   ciudad|
+--------+------------+----+---------+---------+
|       1| 1896 Verano|1896|   Verano|   Athina|
|       2| 1900 Verano|1900|   Verano|    Paris|
|       3| 1904 Verano|1904|   Verano|St. Louis|
|       4| 1906 Verano|1906|   Verano|   Athina|
|       5| 1908 Verano|1908|   Verano|   London|
+--------+------------+----+---------+---------+
only showing top 5 rows



## deporte.csv


In [ ]:
deporteRDD = sc.textFile("deporte.csv").map(lambda line: line.split(","))
deporteRDD = deporteRDD.mapPartitionsWithIndex(eliminaEncabezado)
deporteRDD = deporteRDD.map(lambda l: (
    int(l[0]),
    l[1]
))

schema = StructType((
    StructField("deporte_id", IntegerType(), False),
    StructField("deporte", StringType(), False)
))

deporteDF = sqlContext.createDataFrame(deporteRDD, schema)

In [ ]:
deporteDF.show(5)

+----------+-------------+
|deporte_id|      deporte|
+----------+-------------+
|         1|   Basketball|
|         2|         Judo|
|         3|     Football|
|         4|   Tug-Of-War|
|         5|Speed Skating|
+----------+-------------+
only showing top 5 rows



# Impresión del schema
Es importante ya que a veces no tenemos en un manual estos esquemas

In [ ]:
deporteDF.printSchema()

root
 |-- deporte_id: integer (nullable = false)
 |-- deporte: string (nullable = false)



# Cambiar el nombre de una columna e eliminación de columnas



In [ ]:
deportistaDF.printSchema()

root
 |-- deportista_id: integer (nullable = false)
 |-- nombre: string (nullable = false)
 |-- genero: integer (nullable = false)
 |-- edad: integer (nullable = false)
 |-- altura: integer (nullable = false)
 |-- peso: float (nullable = false)
 |-- equipo_id: integer (nullable = false)



Debido a que los dataFrame son inmutables por eso reasignamos el valor

In [ ]:
deportistaDF = deportistaDF.withColumnRenamed("genero", "sexo").drop("altura")

In [ ]:
deportistaDF.printSchema()

root
 |-- deportista_id: integer (nullable = false)
 |-- nombre: string (nullable = false)
 |-- sexo: integer (nullable = false)
 |-- edad: integer (nullable = false)
 |-- peso: float (nullable = false)
 |-- equipo_id: integer (nullable = false)



#select

Podremos pasar el parámetro de las columnas que nosotros deseemos

## función col
Tiene una diferencia en la forma de procesamiento con los dataFrames ya que cuando nosotros seleccionamos un dataFrame es una operación iterativa, es decir volvemos a cargar todo el dataFrame una y otra vez, por ejemplo en el renombrado y dropeo anterior recorrimnos todo el dataFrame para poderlo eliminar.

Con col generamos una lista en vivo que tendrá todos los valores de las columnas y poder hacer operaciones sobre él que sean pesadas como el renombrado



In [ ]:
deportistaDF = deportistaDF.select("deportista_id", "nombre", col("edad").alias("edadAlJugar"), "equipo_id")

In [ ]:
deportistaDF.show(5)

+-------------+--------------------+-----------+---------+
|deportista_id|              nombre|edadAlJugar|equipo_id|
+-------------+--------------------+-----------+---------+
|            1|           A Dijiang|         24|      199|
|            2|            A Lamusi|         23|      199|
|            3| Gunnar Nielsen Aaby|         24|      273|
|            4|Edgar Lindenau Aabye|         34|      278|
|            5|Christine Jacoba ...|         21|      705|
+-------------+--------------------+-----------+---------+
only showing top 5 rows



Al momento de cargar algunas dataFrames nos dimos cuenta que habían omisión de datos y esto se solucionó agregando ceros para hacer la carga de datos más fácil

Pero si hacemos por ejemplo una selección como la siguiente se nos muestra cierta basura

In [ ]:
deportistaDF.sort("edadAlJugar").show()

+-------------+--------------------+-----------+---------+
|deportista_id|              nombre|edadAlJugar|equipo_id|
+-------------+--------------------+-----------+---------+
|          224|     Mohamed AbdelEl|          0|      308|
|          487|      Inni Aboubacar|          0|      721|
|          226|Sanad Bushara Abd...|          0|     1003|
|           58|    Georgi Abadzhiev|          0|      154|
|          230|    Moustafa Abdelal|          0|      308|
|          102|   Sayed Fahmy Abaza|          0|      308|
|          260|  Ahmed Abdo Mustafa|          0|     1003|
|          139|George Ioannis Abbot|          0|     1043|
|          281|      S. Abdul Hamid|          0|      487|
|          163|     Ismail Abdallah|          0|     1095|
|          285|Talal Hassoun Abd...|          0|      497|
|          173| Mohamed Abdel Fatah|          0|     1003|
|          179|Ibrahim Saad Abde...|          0|     1003|
|          378|     Angelik Abebame|          0|        

# Filtro DataFrame
Para esto haremos un filtro

In [ ]:
deportistaDF = deportistaDF.filter((deportistaDF.edadAlJugar != 0))

In [ ]:
deportistaDF.sort("edadAlJugar").show()

+-------------+--------------------+-----------+---------+
|deportista_id|              nombre|edadAlJugar|equipo_id|
+-------------+--------------------+-----------+---------+
|        71691|  Dimitrios Loundras|         10|      333|
|        70616|          Liu Luyang|         11|      199|
|       118925|Megan Olwen Deven...|         11|      413|
|        52070|        Etsuko Inada|         11|      514|
|        22411|Magdalena Cecilia...|         11|      413|
|        40129|    Luigina Giavotti|         11|      507|
|        47618|Sonja Henie Toppi...|         11|      742|
|        76675|   Marcelle Matthews|         11|      967|
|        37333|Carlos Bienvenido...|         11|      982|
|        51268|      Beatrice Hutiu|         11|      861|
|       126307|        Liana Vicens|         11|      825|
|        48939|             Ho Gang|         12|      738|
|        49142|        Jan Hoffmann|         12|      302|
|        42835|   Werner Grieshofer|         12|       7

# Agrupaciones y operaciones join sobre DF

In [ ]:
deportistaDF.printSchema()

root
 |-- deportista_id: integer (nullable = false)
 |-- nombre: string (nullable = false)
 |-- edadAlJugar: integer (nullable = false)
 |-- equipo_id: integer (nullable = false)



In [ ]:
schema = StructType([
    StructField("deportista_id", IntegerType(), False),
    StructField("nombre", StringType(), False),
    StructField("genero", IntegerType(), False),
    StructField("edad", IntegerType(), False),
    StructField("altura", IntegerType(), False),
    StructField("peso", FloatType(), False),
    StructField("equipo_id", IntegerType(), False)
])
deportistaDF = sqlContext.createDataFrame(deportistaOlimpicoRDD, schema)

In [ ]:
deportistaDF.printSchema()

root
 |-- deportista_id: integer (nullable = false)
 |-- nombre: string (nullable = false)
 |-- genero: integer (nullable = false)
 |-- edad: integer (nullable = false)
 |-- altura: integer (nullable = false)
 |-- peso: float (nullable = false)
 |-- equipo_id: integer (nullable = false)



In [ ]:
resultadoDF.printSchema()

root
 |-- resultado_id: integer (nullable = true)
 |-- medalla: string (nullable = true)
 |-- deportista_id: integer (nullable = true)
 |-- juego_id: integer (nullable = true)
 |-- evento_id: integer (nullable = true)



In [ ]:
juegoDF.printSchema()

root
 |-- juego_id: integer (nullable = false)
 |-- nombre_juego: string (nullable = false)
 |-- anio: string (nullable = false)
 |-- temporada: string (nullable = false)
 |-- ciudad: string (nullable = false)



In [ ]:
deporteDF.printSchema()

root
 |-- deporte_id: integer (nullable = false)
 |-- deporte: string (nullable = false)



## evento.csv
Se carga pero con el nombre deportesOlimpicosDF

In [ ]:
deportesOlimpicosRDD = sc.textFile("evento.csv").map(lambda line: line.split(","))
deportesOlimpicosRDD = deportesOlimpicosRDD.mapPartitionsWithIndex(eliminaEncabezado)
deportesOlimpicosRDD = deportesOlimpicosRDD.map(lambda l: (
    int(l[0]),
    l[1],
    l[2]
))

schema = StructType((
    StructField("evento_id", IntegerType(), False),
    StructField("nombre", StringType(), False),
    StructField("deporte_id", StringType(), False)
))

deportesOlimpicosDF = sqlContext.createDataFrame(deportesOlimpicosRDD, schema)

In [ ]:
deportesOlimpicosDF.show(5)

+---------+--------------------+----------+
|evento_id|              nombre|deporte_id|
+---------+--------------------+----------+
|        1|Basketball Men's ...|         1|
|        2|Judo Men's Extra-...|         2|
|        3|Football Men's Fo...|         3|
|        4|Tug-Of-War Men's ...|         4|
|        5|Speed Skating Wom...|         5|
+---------+--------------------+----------+
only showing top 5 rows



#join
Realizaremos un join, primero indicamos contra que tabla lo vamos a cruzar  

In [ ]:
deportistaDF.join(resultadoDF, deportistaDF.deportista_id == resultadoDF.deportista_id, "left")\
  .join(juegoDF, juegoDF.juego_id == resultadoDF.juego_id, "left")\
  .join(deportesOlimpicosDF,deportesOlimpicosDF.evento_id == resultadoDF.evento_id, "left")\
  .select(deportistaDF.nombre, col("edad").alias("edadAlJugar"),
          "medalla", col("anio").alias("Año de juego"),
          deportesOlimpicosDF.nombre.alias("Nombre de disciplina")).show()

+--------------------+-----------+-------+------------+--------------------+
|              nombre|edadAlJugar|medalla|Año de juego|Nombre de disciplina|
+--------------------+-----------+-------+------------+--------------------+
|Abdul Aziz Hassan...|         23|     NA|        1976|Athletics Men's 2...|
|     Guy A. Abrahams|         23|     NA|        1976|Athletics Men's 2...|
|         Ramli Ahmad|         19|     NA|        1976|Athletics Men's 2...|
|           Hamed Ali|         20|     NA|        1976|Athletics Men's 2...|
|        Joseph Arame|         27|     NA|        1976|Athletics Men's 2...|
|Ainsley Edward Ar...|         19|     NA|        1976|Athletics Men's 2...|
|     Ainsley Bennett|         21|     NA|        1976|Athletics Men's 2...|
|       Ayoub Bodaghi|         28|     NA|        1976|Athletics Men's 2...|
|  Roland Bombardella|         19|     NA|        1976|Athletics Men's 2...|
|      Colin Bradford|         21|     NA|        1976|Athletics Men's 2...|